In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
user_log_df = pd.read_csv("data/user_logs_v2.csv")

In [10]:
transactions_df = pd.read_csv("data/transactions_v2.csv")

In [2]:
train_df = pd.read_csv("data/train_v2.csv")

In [3]:
members_df = pd.read_csv("data/members_v3.csv")

In [ ]:
# Train left join to enrich with member attributes
train_members_df = train_df.merge(
    members_df,
    how="left",
    on="msno"
)
train_members_df.head()


In [20]:
ul_min = user_log_df["date"].min()
ul_max = user_log_df["date"].max()
print(ul_min, ul_max)

20170301 20170331


In [18]:
tr_min = transactions_df["transaction_date"].min()
tr_max = transactions_df["transaction_date"].max()
print(tr_min, tr_max)

20150101 20170331


In [19]:
tr_min2 = transactions_df["membership_expire_date"].min()
tr_max2 = transactions_df["membership_expire_date"].max()
print(tr_min2, tr_max2)

20160419 20361015


In [23]:
user_log_df.info()
user_log_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18396362 entries, 0 to 18396361
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   msno        object 
 1   date        int64  
 2   num_25      int64  
 3   num_50      int64  
 4   num_75      int64  
 5   num_985     int64  
 6   num_100     int64  
 7   num_unq     int64  
 8   total_secs  float64
dtypes: float64(1), int64(7), object(1)
memory usage: 1.2+ GB


,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
count,1.839636e+07,1.839636e+07,1.839636e+07,1.839636e+07,1.839636e+07,1.839636e+07,1.839636e+07,1.839636e+07
mean,2.017032e+07,6.191401e+00,1.508789e+00,9.413759e-01,1.079905e+00,3.028246e+01,2.903615e+01,7.904814e+03
std,8.916720e+00,1.342827e+01,3.908539e+00,1.924840e+00,3.518409e+00,4.203641e+01,3.219866e+01,1.013632e+04
min,2.017030e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e-03
25%,2.017031e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,8.000000e+00,1.959944e+03
50%,2.017032e+07,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.700000e+01,1.800000e+01,4.582990e+03
75%,2.017032e+07,7.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,3.700000e+01,3.800000e+01,9.848441e+03
max,2.017033e+07,5.639000e+03,9.120000e+02,5.080000e+02,1.561000e+03,4.110700e+04,4.925000e+03,9.194059e+06


In [22]:
train_df.info()
train_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970960 entries, 0 to 970959
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   msno      970960 non-null  object
 1   is_churn  970960 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 14.8+ MB


,is_churn
count,970960.000000
mean,0.089942
std,0.286099
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [4]:
T = pd.Timestamp("2017-04-01")

## EPIC 3

In [5]:
# train <- member LEFT JOIN
train_members_df = train_df.merge(members_df, on="msno", how="left", validate="1:1")

# sanity check: row 수 절대 변하면 안 됨
assert len(train_members_df) == len(train_df), "Row count changed after join! (should not happen)"
print("Joined shape:", train_members_df.shape)

Joined shape: (970960, 7)


In [6]:
# 결측치 현황
missing_rate = train_members_df.isna().mean().sort_values(ascending=False)
print(missing_rate)

# 타겟 분포도 같이 확인
print(train_members_df["is_churn"].value_counts(normalize=True))

gender                    0.599463
city                      0.113283
bd                        0.113283
registered_via            0.113283
registration_init_time    0.113283
msno                      0.000000
is_churn                  0.000000
dtype: float64
is_churn
0    0.910058
1    0.089942
Name: proportion, dtype: float64


In [7]:
# registration_init_time → 날짜로 변환 + 가입 후 경과일 파생
train_members_df["registration_init_time"] = pd.to_datetime(
    train_members_df["registration_init_time"],
    format="%Y%m%d",
    errors="coerce",
)

train_members_df["days_since_registration"] = (T - train_members_df["registration_init_time"]).dt.days

# sanity check
print(train_members_df[["registration_init_time", "days_since_registration"]].describe())

              registration_init_time  days_since_registration
count                         860967            860967.000000
mean   2013-09-13 07:10:23.952834560              1295.701112
min              2004-03-26 00:00:00               -23.000000
25%              2012-02-14 00:00:00               439.000000
50%              2014-06-02 00:00:00              1034.000000
75%              2016-01-18 00:00:00              1873.000000
max              2017-04-24 00:00:00              4754.000000
std                              NaN              1097.536149


In [8]:
# -23 = T 이후 가입자
# 따라서 nan처리

In [9]:
# 음수 경과일은 의미 없으므로 NaN 처리
train_members_df.loc[train_members_df["days_since_registration"] < 0, "days_since_registration"] = np.nan

# sanity check
print(train_members_df["days_since_registration"].describe())
print("negative count:",
      (train_members_df["days_since_registration"] < 0).sum())

count    860966.000000
mean       1295.702643
std        1097.535867
min           1.000000
25%         439.000000
50%        1034.000000
75%        1873.000000
max        4754.000000
Name: days_since_registration, dtype: float64
negative count: 0


In [10]:
# bd(나이) 이상치 처리 -> 유효 범위: 10 ~ 80, 그 외 np.nan
train_members_df["bd"] = pd.to_numeric(train_members_df["bd"], errors="coerce")

valid_age_min, valid_age_max = 10, 80
train_members_df.loc[(train_members_df["bd"] < valid_age_min) | (train_members_df["bd"] > valid_age_max), "bd"] = np.nan

# 처리 결과 확인
print(train_members_df["bd"].describe())
print("bd missing rate:", train_members_df["bd"].isna().mean())

count    386393.000000
mean         29.876853
std           8.797025
min          10.000000
25%          24.000000
50%          28.000000
75%          34.000000
max          80.000000
Name: bd, dtype: float64
bd missing rate: 0.6020505479113455


In [11]:
# gender 결측치 -> "unknown"
train_members_df["gender"] = train_members_df["gender"].astype("object").fillna("unknown")
print(train_members_df["gender"].value_counts(dropna=False))

gender
unknown    582055
male       204561
female     184344
Name: count, dtype: int64


In [12]:
train_members_df["city"] = train_members_df["city"].astype("Int64")
train_members_df["registered_via"] = train_members_df["registered_via"].astype("Int64")

# 결측이 있다면 unknown 코드(-1)로 채우는 방식(선택)
train_members_df["city"] = train_members_df["city"].fillna(-1)
train_members_df["registered_via"] = train_members_df["registered_via"].fillna(-1)

train_members_df["city"] = train_members_df["city"].astype("int32")
train_members_df["registered_via"] = train_members_df["registered_via"].astype("int32")

In [13]:
# 결측 비율 확인
(train_members_df.isna().mean()
 .sort_values(ascending=False)
 .head(10))

bd                         0.602051
days_since_registration    0.113284
registration_init_time     0.113283
msno                       0.000000
is_churn                   0.000000
city                       0.000000
gender                     0.000000
registered_via             0.000000
dtype: float64

## 요약

```
train_members_df
├─ 타겟: is_churn (완전)
├─ 정적 정보: city, registered_via (결측 처리 완료)
├─ 성별: gender ('unknown' 처리)
├─ 나이: bd (이상치 제거 → NaN)
├─ 가입 정보: days_since_registration (음수 제거 → NaN)
```

- 결측 처리만 했고, 결측값을 대체하는 것은 아직 정하지 않았음

In [14]:
# 지금 단계의 베이스 테이블 저장
train_members_df.to_parquet("train_members_base.parquet", index=False)

print("Saved: train_members_base.parquet")
print(train_members_df.head())

Saved: train_members_base.parquet
                                           msno  is_churn  city    bd  \
0  ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=         1     5  28.0   
1  f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=         1    13  20.0   
2  zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=         1    13  18.0   
3  8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=         1     1   NaN   
4  K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=         1    13  35.0   

    gender  registered_via registration_init_time  days_since_registration  
0     male               3             2013-12-23                   1195.0  
1     male               3             2013-12-23                   1195.0  
2     male               3             2013-12-27                   1191.0  
3  unknown               7             2014-01-09                   1178.0  
4   female               7             2014-01-25                   1162.0  


In [4]:
df = pd.read_parquet("train_members_base.parquet")
print(df.shape)
df.head()

(970960, 8)


,msno,is_churn,city,bd,gender,registered_via,registration_init_time,days_since_registration
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5,28.0,male,3,2013-12-23,1195.0
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13,20.0,male,3,2013-12-23,1195.0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13,18.0,male,3,2013-12-27,1191.0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,1,NaN,unknown,7,2014-01-09,1178.0
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13,35.0,female,7,2014-01-25,1162.0


In [5]:
df.dtypes

msno                               object
is_churn                            int64
city                                int32
bd                                float64
gender                             object
registered_via                      int32
registration_init_time     datetime64[ns]
days_since_registration           float64
dtype: object

In [6]:
(df.isna().mean()
 .sort_values(ascending=False)
 .head(10))

bd                         0.602051
days_since_registration    0.113284
registration_init_time     0.113283
msno                       0.000000
is_churn                   0.000000
city                       0.000000
gender                     0.000000
registered_via             0.000000
dtype: float64

In [7]:
df["is_churn"].value_counts(normalize=True)

is_churn
0    0.910058
1    0.089942
Name: proportion, dtype: float64

In [8]:
pd.crosstab(
    df["gender"].isna(),
    df["is_churn"],
    normalize="index"
)

is_churn,0,1
gender,,
False,0.910058,0.089942


In [9]:
pd.crosstab(
    df["gender"],
    df["is_churn"],
    normalize="index"
)

is_churn,0,1
gender,,
female,0.870134,0.129866
male,0.870963,0.129037
unknown,0.936442,0.063558


In [2]:
df = pd.read_csv("data/processed/transactions_agg.csv")

df.describe()

,days_since_last_payment,last_plan_days,last_payment_method,total_payment_count,avg_amount_per_payment,subscription_months_est
count,1.186674e+06,1.186674e+06,1.186674e+06,1.186674e+06,1.186674e+06,1.186674e+06
mean,3.670745e+01,7.084151e+01,3.781197e+01,1.158291e+00,3.056367e+02,2.624120e+00
std,7.065211e+01,1.087215e+02,5.017739e+00,1.092789e+00,4.662365e+02,3.967828e+00
min,1.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.000000e+00,3.000000e+01,3.600000e+01,1.000000e+00,9.900000e+01,1.000000e+00
50%,1.700000e+01,3.000000e+01,4.000000e+01,1.000000e+00,1.490000e+02,1.000000e+00
75%,2.700000e+01,3.000000e+01,4.100000e+01,1.000000e+00,1.490000e+02,1.000000e+00
max,8.200000e+02,4.500000e+02,4.100000e+01,7.600000e+01,2.000000e+03,2.445000e+02


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186674 entries, 0 to 1186673
Data columns (total 7 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   msno                     1186674 non-null  object 
 1   days_since_last_payment  1186674 non-null  int64  
 2   last_plan_days           1186674 non-null  int64  
 3   last_payment_method      1186674 non-null  int64  
 4   total_payment_count      1186674 non-null  int64  
 5   avg_amount_per_payment   1186674 non-null  float64
 6   subscription_months_est  1186674 non-null  float64
dtypes: float64(2), int64(4), object(1)
memory usage: 63.4+ MB


In [4]:
df.columns

Index(['msno', 'days_since_last_payment', 'last_plan_days',
       'last_payment_method', 'total_payment_count', 'avg_amount_per_payment',
       'subscription_months_est'],
      dtype='object')